In [1]:
import os
import time
import gzip
import numpy as np
import pickle
import msprime
import moments
import demes
import ray

assert msprime.__version__ >= "1"

if not os.path.isdir("./data/"):
    os.makedirs("./data/")
os.system("rm ./data/*.vcf.gz")
os.system("rm ./data/*.h5")

rm: cannot remove './data/*.vcf.gz': No such file or directory
rm: cannot remove './data/*.h5': No such file or directory


256